In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
import torchvision.models as models
import time

In [ ]:
data_path = os.getcwd()
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(size=(224, 224)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
batch_size = 64

trainset = torchvision.datasets.CIFAR10(root= data_path, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root= data_path, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
train_steps = len(trainloader.dataset) // batch_size
test_steps = len(testloader.dataset) // batch_size

print(f'Train steps:{train_steps}\nTest Steps:{test_steps}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
vgg19 = models.vgg19(pretrained=False, progress=True,num_classes = 10)

In [ ]:
learning_rate = 0.0025
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg19.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg19 = vgg19.to(device)
print(device)

In [ ]:
H = {
    "train_loss": [],
    "train_acc": [],
    "test_loss": [],
    "test_acc": []
}

In [ ]:
EPOCHS = 30
startTime = time.time()
# loop over our epochs
for e in range(0, EPOCHS):
    # set the model in training mode
    vgg19.train()
    # initialize the total training and validation loss
    total_train_loss = 0
    total_test_loss = 0
    # initialize the number of correct predictions in the training
    # and validation step
    train_correct = 0
    test_correct = 0
    # loop over the training set
    for (x, y) in trainloader:
        # send the input to the device
        (x, y) = (x.to(device), y.to(device))
        # perform a forward pass and calculate the training loss
        pred = vgg19(x)
        loss = criterion(pred, y)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        total_train_loss += loss
        train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # switch off autograd for evaluation
    with torch.no_grad():
        # set the model in evaluation mode
        vgg19.eval()
        # loop over the validation set
        for (x, y) in testloader:
            # send the input to the device
            (x, y) = (x.to(device), y.to(device))
            # make the predictions and calculate the validation loss
            pred = vgg19(x)
            total_test_loss += criterion(pred, y)
            # calculate the number of correct predictions
            test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # calculate the average training and validation loss
    avg_train_loss = total_train_loss / train_steps
    avg_test_loss = total_test_loss / test_steps
    # calculate the training and validation accuracy
    train_correct = train_correct / len(trainloader.dataset)
    test_correct = test_correct / len(testloader.dataset)
    # update our training history
    H["train_loss"].append(avg_train_loss.cpu().detach().numpy())
    H["train_acc"].append(train_correct)
    H["test_loss"].append(avg_test_loss.cpu().detach().numpy())
    H["test_acc"].append(test_correct)
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
        avg_train_loss, train_correct))
    print("Val loss: {:.6f}, Test accuracy: {:.4f}\n".format(
        avg_test_loss, test_correct))
    
endTime = time.time()
print('Finished training')
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

In [ ]:
import matplotlib.pyplot as plt
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()

plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["test_acc"], label="test_acc")
plt.title("Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig('./accuracy.png')


In [ ]:
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig('./loss.png')

In [ ]:
torch.save(vgg19.state_dict(), './vgg19/tomas_vgg19.pth')

In [ ]:
# device = torch.device('cpu')
# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH, map_location=device))